# Palmer penguins data

In [ ]:
! wget "https://raw.githubusercontent.com/mcnakhaee/palmerpenguins/master/palmerpenguins/data/penguins.csv"

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colormaps

In [ ]:
penguins = pd.read_csv("penguins.csv")

In [ ]:
penguins.head()

In [ ]:
penguins["species"].unique() # There are three unique species

The bill length and bill depth of members of one species are likely to be similar, and different from those of the members of another species. It is reasonable to expect that we will see three cluster, one for each species, if we plot these two features.

In [ ]:
pd.plotting.scatter_matrix(penguins[["bill_length_mm","bill_depth_mm","flipper_length_mm","body_mass_g"]], figsize=(8,8))

# Mean shift clustering

We will use a Gaussian kernel for mean shift clustering. The implementation is written using PyTorch so that we can run this on a GPU.

In [ ]:
import torch

In [ ]:
def gaussian(x, mu, sigma):
    """Calculates the Gaussian function:

    .. math::

    g(x) = \frac{1}{\sigma\sqrt{2\pi}}\exp\left(-\frac{1}{2}\frac{(x - \mu)^2}{\sigma^2}\right)

    Args:
        x (torch.Tensor): Points at which the value of the Gaussian will be calculated.
        mu (float): Mean of the Gaussian.
        sigma (float): Standard deviation of the Gaussian.
    """
    return torch.exp(-0.5 * ((x - mu) / sigma) ** 2) / (
        sigma * torch.sqrt(torch.tensor(2 * torch.pi))
    )

In [ ]:
def meanshift(data, bw, bs=None, epochs=10):
    """Determines the centroids of clusters using the mean shift algorithm.

    Args:
        data (torch.Tensor): Data whose centroids are to be determined.
        bw (float): Bandwidth, or standard deviation, of the Gaussian kernel.
        bs (int, None): Size of the batch. If `None` or larger than `len(data)`, then set
                        to `len(data)`.
        epochs (int): Number of training iterations. Default value is 10.
    """
    X = data.clone()
    n = len(data)
    if bs is None or bs > n:
        bs = n
    for _ in range(epochs):
        for i in range(0, n, bs):
            dist = torch.cdist(X[i : min(i + bs, n)], X)
            weights = gaussian(dist, 0, bw)
            X[i : min(i + bs, n)] = (weights @ X) / weights.sum(1, keepdim=True)
    return X

In [ ]:
data = penguins[["bill_length_mm", "bill_depth_mm"]]
data

In [ ]:
data = data.dropna()
data

In [ ]:
data = torch.tensor(data.values)
if torch.cuda.is_available(): data = data.cuda()

In [ ]:
type(data)

In [ ]:
centroids = meanshift(data, 1.5, 8).cpu()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(data = penguins, x = "bill_length_mm", y = "bill_depth_mm")
ax.set_xlabel("bill_length_mm")
ax.set_ylabel("bill_depth_mm")
ax.scatter(centroids[:, 0], centroids[:, 1], marker="x", color="red")